<a href="https://colab.research.google.com/github/angel-palkina/angel-palkina/blob/main/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%86%D0%B8%D1%84%D1%80_%D0%B2_%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%82%D0%B2%D0%B5_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Свёрточная нейросеть с картами активации
Cоnvolutional - свёрточная
Pooling - сжатие
Вывод в слой классификации

Сменили среду выполнения на GPU - видеокарту


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential # сети последовательного распространения
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization #полносвязный слой, свёрточный слой работающий с 2мерным изовражением, слой для сжатия, слой выпрямления матриц, слой пряпятствия переобучению нейросети, слой нормализации пакета данных(уменьшения больших значений нейронов для увеличения скорости обучения)
from tensorflow.keras.datasets import mnist #картинки для обучения
from tensorflow.keras import utils #чтобы привести выходные данные в one hot encoding
from tensorflow.keras.preprocessing import image # для подгрузки тестовой фотографии с компьютера

In [ ]:
(x_train,y_train),(x_test, y_test) = mnist.load_data()#нужны наборы для обучения, валидации(тест для перестраивания процесса обучения), а также тестовый набор 


In [ ]:
x_train = x_train/255 # преобразование картинок в массив со значениями от 0 до 1
x_train = x_train.reshape(60000,28,28,1)

In [ ]:
x_test = x_test/255 # преобразование картинок в массив со значениями от 0 до 1
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
y_train = utils.to_categorical(y_train,10)
y_train[0] #разбрасывания правильного ответа на 10 нейронов

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [ ]:
model = Sequential()#сознание модели нейронной сети
model.add(BatchNormalization (input_shape = (28, 28, 1)))#создание первого скрытого нормализовочного нейронного слоя c указанием кол-ва входных shape нейронов
model.add(Conv2D(256, 3, padding = 'same', activation= 'relu'))#создание скрытого свёрточного нейронного слоя 
model.add(Conv2D(256, 3, padding = 'same', activation= 'relu'))# 1 первый параметр кол-во ядер, 2 - размер стороны ядра
model.add(MaxPooling2D(pool_size= (2,2)))# параметр - размер сжатия, сжатие происходит по максимальному значению 
model.add(Dropout(0.25))# создание скрытого свёрточного нейронного слоя, который отключает определённый % нейронов для избежения запоминания картинок
model.add(Conv2D(128, 3, padding = 'same', activation= 'relu'))
model.add(Conv2D(128, 3, padding = 'same', activation= 'relu'))
model.add(MaxPooling2D(pool_size= (2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, 3, padding = 'same', activation= 'relu'))
model.add(Conv2D(64, 3, padding = 'same', activation= 'relu'))
model.add(Flatten())
model.add(Dense(1024, activation= 'relu'))#создание скрытого полносвязного нейронного слоя
model.add(Dense(1024, activation= 'relu'))
model.add(Dense(512, activation= 'relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation= 'softmax')) #выходной слой на 10 нейронов

In [ ]:
model.summary() #вывод архитектура модели

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics='accuracy')# какая функция ошибки, какой метод градиентного спуска, какая метрика

In [ ]:
model.fit(x_train, y_train, batch_size= 150, epochs=10, validation_split = 0.2)#размер batch - это кол-во примеров, которое мы поссылаем в нейронную сеть за 1 раз, эпоха - сколько раз мы проведём наши картинки пройдут через нейронную сеть, 0.2 - %, 

Epoch 1/10
320/320 [==============================] - 39s 80ms/step - loss: 0.2743 - accuracy: 0.9081 - val_loss: 0.7363 - val_accuracy: 0.9715
Epoch 2/10
320/320 [==============================] - 26s 82ms/step - loss: 0.0634 - accuracy: 0.9814 - val_loss: 0.0488 - val_accuracy: 0.9871
Epoch 3/10
320/320 [==============================] - 25s 79ms/step - loss: 0.0459 - accuracy: 0.9864 - val_loss: 0.0525 - val_accuracy: 0.9866
Epoch 4/10
320/320 [==============================] - 25s 78ms/step - loss: 0.0357 - accuracy: 0.9898 - val_loss: 0.0460 - val_accuracy: 0.9873
Epoch 5/10
320/320 [==============================] - 26s 80ms/step - loss: 0.0336 - accuracy: 0.9898 - val_loss: 0.0387 - val_accuracy: 0.9886
Epoch 6/10
320/320 [==============================] - 25s 78ms/step - loss: 0.0283 - accuracy: 0.9915 - val_loss: 0.0366 - val_accuracy: 0.9915
Epoch 7/10
320/320 [==============================] - 25s 78ms/step - loss: 0.0234 - accuracy: 0.9934 - val_loss: 0.0496 - val_accuracy:

In [ ]:
k = 0
prediction = model.predict(x_test) #предсказание тестовой картинки с номером k
print('На картинке цифра:',np.argmax(prediction[k]))

На картинке цифра: 7


In [ ]:
p = 0
for i in range(10000):  #цикл подсчёта кол-ва правильных результатов
  if np.argmax(prediction[i]) == y_test[i]:
    p += 1
print(p/x_test.shape[0]*100, '% - вероятность правильности предсказания')

99.09 % - вероятность правильности предсказания
